In [113]:
# imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

pd.options.plotting.backend = "plotly"

from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet import Prophet
import itertools
from sktime.performance_metrics.forecasting import (
    mean_absolute_scaled_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
)
import time
import holidays
import datetime


# define functions

In [294]:
def preprocessing_endo(ts, steps_ahead=30, split=True):
    fb_df = ts[["sales"]]
    fb_df["sales"] = ts["sales"] / 1e6
    fb_df = fb_df.reset_index().rename({"date": "ds", "sales": "y"}, axis=1)

    if split:
        fb_train = fb_df.iloc[:-steps_ahead]
        fb_test = fb_df.iloc[-steps_ahead:]
        return {"y_train": fb_train, "y_test": fb_test}
    else:
        return fb_df


def preprocessing_exog(df_exog):
    df_promo = pd.DataFrame({
        "holiday": "promo_day",
        "ds": df_exog[df_exog["promo_day"] == 1]["promo_day"].index})
    df_off = pd.DataFrame({
        "holiday": "off_day",
        "ds": df_exog[df_exog["off_day"] == 1]["off_day"].index})
    return pd.concat([df_promo, df_off])


# grid seach Prophet
def grid_search_prophet(prophet_data_endo, prophet_data_exog, param_grid):
    # create df to store result
    mape_PROPHET = pd.DataFrame(columns=["params", "mape"])
    
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

    # grid search
    y_train = prophet_data_endo['y_train']
    y_test = prophet_data_endo['y_test']
    for params in all_params:
        m = Prophet(holidays=prophet_data_exog,
                    **params).fit(y_train)  # Fit model with given params
        y_hat = m.predict(y_test)['yhat']
        mape_PROPHET = mape_PROPHET.append(
            {"params": params, 
             "mape": round(mean_absolute_percentage_error(y_test['y'], y_hat), 3)}, ignore_index=True)
    mape_PROPHET.sort_values('mape', inplace=True)
    return mape_PROPHET


# CV 
def cross_validation_result(data, model, model_name, rolls=4, horizon=30):
    """ """
    mae_CVs = []
    rmse_CVs = []
    mape_CVs = []
    mase_CVs = []
    for i in range(rolls):
        print(f"fold {i}---------------")
        
        # split data
        y_train = data.iloc[: -(rolls - i) * horizon]
        y_test = data.iloc[
            np.r_[-(rolls - i) * horizon : -(rolls - i - 1) * horizon]]

        # fit model
        model.fit(y_train)
        
        # make forecast
        y_hat = model.predict(y_test)

        # score
        mae_CVs.append(round(mean_absolute_error(y_test['y'], y_hat['yhat']), 3))
        rmse_CVs.append(round(mean_squared_error(y_test['y'], y_hat['yhat'], square_root=True), 3))
        mape_CVs.append(round(mean_absolute_percentage_error(y_test['y'], y_hat['yhat']), 3))
        mase_CVs.append(round(mean_absolute_scaled_error(y_test['y'], y_hat['yhat'], y_train=y_train['y']), 3))
    
    return {'store':model_name,
           'mae_PROPHET':np.mean(mae_CVs),
           'rmse_PROPHET':np.mean(rmse_CVs),
           'mape_PROPHET':np.mean(mape_CVs),
           'mase_PROPHET':np.mean(mase_CVs),
           'fc_PROPHET':y_hat,
           }



# read data

In [139]:
df_store = pd.read_pickle("data/df_daily.pkl")
df_company = df_store.groupby("date").sum()[["sales"]]
df_exog = pd.read_pickle("data/df_exog.pkl")

df_company_pro = preprocessing_endo(df_company, split=True)
df_exog_pro = preprocessing_exog(df_exog)


# tune on company data

In [171]:
# define param grid
param_grid = {
    "changepoint_prior_scale": [0.005, 0.01],#, 0.05, 0.1],
    "seasonality_prior_scale": [1.0],#[0.01, 0.1, 1.0],
    # "daily_seasonality": [False],
    # "weekly_seasonality": [True],
    # "yearly_seasonality": [True],
}



## grid search
- fit once per param set on train set
- predict & score on test set (no CV)

In [183]:
grid_search_result = grid_search_prophet(df_company_pro, df_exog_pro, param_grid)


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -11.5961
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      82        2696.5   0.000253099        715.08   4.158e-07       0.001      139  LS failed, Hessian reset 
      99       2696.67   0.000213371       764.172           1           1      163   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     134       2696.77   5.62889e-05       906.035   6.709e-08       0.001      251  LS failed, Hessian reset 
     160        2696.8   4.24363e-07       800.308   6.061e-10       0.001      317  LS failed, Hessian reset 
     169        2696.8   8.75066e-10       559.143   0.0005954      0.4386      333   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/1829206763.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/Users/giangphan23

Initial log joint probability = -11.5961
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2716.57   2.66253e-05       748.001           1           1      143   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     122       2717.06   0.000104735        655.65   1.424e-07       0.001      210  LS failed, Hessian reset 
     173       2717.12   8.65058e-09       608.607      0.4394      0.4394      279   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/1829206763.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/Users/giangphan23

...

Initial log joint probability = -11.5961
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       2755.14   0.000142571       81.7021      0.6235      0.6235      113   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     169       2755.96   0.000148644       56.1762   4.037e-06       0.001      232  LS failed, Hessian reset 
     196       2755.98   0.000101308       43.0333     2.4e-06       0.001      298  LS failed, Hessian reset 
     199       2755.98    2.8701e-05       34.1259           1           1      301   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     252       2756.06   9.46055e-05       58.0404   1.812e-06       0.001      420  LS failed, Hessian reset 
     299       2756.14   2.12645e-06        33.881      0.3332      0.3332      480   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Note

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/1829206763.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/Users/giangphan23

Initial log joint probability = -11.5961
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        2755.4     0.0028438       91.9399           1           1      114   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2756.13    0.00073735       58.9274           1           1      252   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     202       2756.15   0.000496193       61.1436   8.504e-06       0.001      291  LS failed, Hessian reset 
     299       2756.37   2.21789e-05       32.3233      0.1857           1      417   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     371       2756.38   3.44151e-07       36.7436      0.1854           1      518   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/var/folders/kf/st4z7vm96nz8c7npjz8kjxt40000gp/T/ipykernel_77040/1829206763.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mape_PROPHET = mape_PROPHET.append(


,params,mape
0,"{'changepoint_prior_scale': 0.005, 'seasonalit...",0.196
1,"{'changepoint_prior_scale': 0.005, 'seasonalit...",0.194
2,"{'changepoint_prior_scale': 0.005, 'seasonalit...",0.193
3,"{'changepoint_prior_scale': 0.01, 'seasonality...",0.227
4,"{'changepoint_prior_scale': 0.01, 'seasonality...",0.21
5,"{'changepoint_prior_scale': 0.01, 'seasonality...",0.214
6,"{'changepoint_prior_scale': 0.05, 'seasonality...",0.211
7,"{'changepoint_prior_scale': 0.05, 'seasonality...",0.205
8,"{'changepoint_prior_scale': 0.05, 'seasonality...",0.205
9,"{'changepoint_prior_scale': 0.1, 'seasonality_...",0.196


In [221]:
grid_search_result


,params,mape
2,"{'changepoint_prior_scale': 0.005, 'seasonalit...",0.193
11,"{'changepoint_prior_scale': 0.1, 'seasonality_...",0.193
1,"{'changepoint_prior_scale': 0.005, 'seasonalit...",0.194
10,"{'changepoint_prior_scale': 0.1, 'seasonality_...",0.194
0,"{'changepoint_prior_scale': 0.005, 'seasonalit...",0.196
9,"{'changepoint_prior_scale': 0.1, 'seasonality_...",0.196
7,"{'changepoint_prior_scale': 0.05, 'seasonality...",0.205
8,"{'changepoint_prior_scale': 0.05, 'seasonality...",0.205
4,"{'changepoint_prior_scale': 0.01, 'seasonality...",0.21
6,"{'changepoint_prior_scale': 0.05, 'seasonality...",0.211


In [214]:
tuned_prophet_params = grid_search_result.iloc[0, 0]
tuned_mape = grid_search_result.iloc[0, 1]
tuned_mape, tuned_prophet_params


(0.193, {'changepoint_prior_scale': 0.005, 'seasonality_prior_scale': 1.0})

# fit on store data

In [295]:
all_stores_result_CV = []
for store in df_store["store_id"].unique():#[:2]:
    print(f"processing stores {store}...\n")
    model_name = "store_" + str(store)

    # data
    df_1_store = df_store[df_store["store_id"] == store].set_index("date")[["sales"]]
    df_1_store_pro = preprocessing_endo(df_1_store, split=False)

    # define model
    model_store = Prophet(**tuned_prophet_params, holidays=df_exog_pro)
    
    # score model with CV on store data
    cv_score = cross_validation_result(df_1_store_pro, model_store, model_name=model_name)

    # result
    all_stores_result_CV.append(cv_score)
all_stores_result_CV = pd.DataFrame(all_stores_result_CV)


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


processing stores 307222...

fold 0---------------


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -7.24017
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      78       2533.69    7.4264e-06       451.908   1.139e-08       0.001      143  LS failed, Hessian reset 
      99        2533.7   1.63124e-06       488.537           1           1      173   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       2534.43   5.79594e-07       624.293      0.7517      0.7517      309   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     226       2534.43   8.95266e-07       737.298   1.456e-09       0.001      390  LS failed, Hessian reset 
     243       2534.43   2.17023e-09       665.438      0.1032      0.1032      415   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


...

/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


fold 2---------------


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


fold 3---------------


/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/giangphan23/OneDrive - Norges Handelshøyskole/Thesis/Sales_forecast/ap8venv/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


# result

In [299]:
# all_stores_result_CV.to_pickle("results/f8/PROPHET.pkl")


In [297]:
all_stores_result_CV.head()


,store,mae_PROPHET,rmse_PROPHET,mape_PROPHET,mase_PROPHET,fc_PROPHET
0,store_307222,10.68525,13.63550,0.36600,1.09275,ds trend yhat_lower yhat_upp...
1,store_307244,8.74925,10.94050,0.39975,1.12800,ds trend yhat_lower yhat_upp...
2,store_307248,6.36175,8.12425,0.38300,0.92750,ds trend yhat_lower yhat_upp...
3,store_320264,5.93650,7.95275,0.44100,0.86750,ds trend yhat_lower yhat_uppe...
4,store_328165,26.14350,36.35825,0.41375,1.22200,ds trend yhat_lower yhat_upp...


In [298]:
all_stores_result_CV.describe()


,mae_PROPHET,rmse_PROPHET,mape_PROPHET,mase_PROPHET
count,38.000000,38.000000,38.000000,38.000000
mean,8.129079,10.686875,0.480375,1.112441
std,5.842856,8.209956,0.120029,0.229924
min,2.722500,3.459250,0.323500,0.811000
25%,4.373250,5.575813,0.386188,0.953375
50%,6.065125,8.168625,0.446750,1.080875
75%,8.877437,11.513875,0.548250,1.183000
max,27.480000,38.433750,0.764500,2.017500
